## Next word prediction

Quick example code to obtain next word predictions of a sample of text.

First load the needed libraries

In [1]:
!pip install together

import string
from together import Together
import math

/Users/bg/PycharmProjects/next-word-pred-LLMWS2025/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Write some text. 

In [4]:
prompt = "The mouse eats the cheese."

Prepare the text

In [5]:
text_list = prompt.split()
pred_w = [0] * len(text_list)
i_list = [0] * len(text_list)

key="X"


Define function for next token prediction

In [7]:
def next_token_probs(prompt: str, key: str):
    client = Together(api_key=key)

    response = client.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct-Turbo",
        prompt=prompt,
        max_tokens=1,           # Only predict the next token
        logprobs=5,         # Get top-k most likely tokens
    )

     # Get the LogprobsPart object for the first (and only) token
    logprobs_part = response.choices[0].logprobs

    # top_logprobs is a list of dicts (one dict per token)
    top_logprobs_dict = logprobs_part.top_logprobs[0]  # first token only

    # Convert logprobs to probabilities
    results = [(token, round(math.exp(lp), 4)) for token, lp in top_logprobs_dict.items()]

    return results

Run a loop for each word of the sentence. To calculate the 5 most likely next words.

Output: List of word/probability pairs 

In [10]:
i = 0
for i in range(0,len(text_list)-1):
        if len(text_list[i].translate(str.maketrans('', '', string.punctuation)).replace("\n", "")) > 0:
            prompt_text = " ".join(text_list[:i+1])
            print(prompt_text)
            print(next_token_probs(prompt_text, key))

The
[('#', 0.0008), ('’s', 0.0008), (' own', 0.0008), ("'s", 0.0007), (' importance', 0.0007)]
The mouse
[(' is', 0.061), (' was', 0.0419), ("'s", 0.0307), (' wheel', 0.0254), (' has', 0.0211)]
The mouse eats
[(' the', 0.1463), (' a', 0.114), (' ', 0.0736), (' food', 0.0288), (' its', 0.0288)]
The mouse eats the
[(' cheese', 0.2096), (' seeds', 0.0413), (' food', 0.0321), (' apple', 0.0284), (' cookie', 0.0183)]
